# Project 001 UCSD Data Science Visualization Bootcamp
* Noreen Malik 😃
* Rene Venema("dutch") 🐱
* John Warlop 🦁
* Alex Wooten 🏄 Adding my comment to test

😛😳😗😓🙉😸🙈🙊😽💀💢💥✨💏👫👄👃👀👛👛🗼🔮🔮🎄🎅👻

## Dependencies -- imports


In [5]:
import pandas as pd
import json
# john branch 7:31 pm

## Constants -- API Keys, etc

In [13]:
# Read in API(.gitignore file has entry for api_keys.json file)
IRS_DATAFRAME_PICKLE = 'df_irs.pkl'
with open('api_keys.json') as json_data:
    api_keys = json.load(json_data)
#print(api_keys['socrata']['App_Token'])
#print(api_keys['socrata']['Secret_Token'])
#print(api_keys['zillow']['ZWSID'])


## Functions 


In [9]:
def winning_team():
    print("Noreen, 'dutch', Alex and John make for a winning team")
    
def return_irs_df(file_name):
    #return the pickled dataframe for irs data
    df_irs = pd.read_pickle(file_name)
    return(df_irs)

## Solution Space


In [10]:
winning_team()

Noreen, 'dutch', Alex and John make for a winning team


In [11]:
# Test if return_irs_df works
df_irs = return_irs_df(IRS_DATAFRAME_PICKLE)
df_irs.head()

,ST,Zip,AGI,TtlR,TtlSR,TtlJR,Eld
0,AL,0,1,836320.0,481570.0,109790.0,149910.0
1,AL,0,2,494830.0,206630.0,146250.0,107480.0
2,AL,0,3,261250.0,80720.0,139280.0,73400.0
3,AL,0,4,166690.0,28510.0,124650.0,49990.0
4,AL,0,5,212660.0,19520.0,184320.0,65870.0


# Create dataframe from csv file(dutch)

# Avg Cost Per Provide (Alex)

# Avg Cost of Procedure in each state(John)

# Comparing procedure costs vs income in zip code(John)